In [2]:
#!pip install transformers

In [3]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from tqdm.notebook import tqdm

Using TensorFlow backend.


In [4]:
%%time
df = pd.read_csv('/content/drive/My Drive/Data/Jigsaw Multilingual Toxic Comment Classification/jigsaw-toxic-comment-train.csv')

CPU times: user 1.09 s, sys: 143 ms, total: 1.24 s
Wall time: 22.9 s


In [5]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [6]:
def get_inputs(text, step):
    input_ids = []
    attention_mask = []
    for i in tqdm(range(0, len(text), step)):
        end = i + step
        if len(text) - i < step:
            end = len(text)
        tokenized = np.asarray([tokenizer.encode(x, add_special_tokens=True, max_length = 128, truncation = True) for x in text[i:end]])
        max_len = max([len(x) for x in tokenized])
        curr_input_ids = pad_sequences(tokenized, maxlen = max_len, padding = 'post')
        curr_attention_mask = np.where(curr_input_ids != 0, 1, 0)
        
        input_ids.extend(curr_input_ids)
        attention_mask.extend(curr_attention_mask)
        
    return np.asarray(input_ids), np.asarray(attention_mask)

In [7]:
%%time
train_input_ids, train_attention_mask = get_inputs(df['comment_text'], 128)
y_train = df['toxic'].values


CPU times: user 3min 53s, sys: 1.86 s, total: 3min 55s
Wall time: 3min 56s


In [8]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    sequence_output = transformer(input_word_ids, attention_mask = attention_mask)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=[input_word_ids, attention_mask], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
%%time
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.37.247.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.37.247.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [17]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=128)

- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [23]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_1 (TFDisti ((None, 128, 768),)  134734080   input_word_ids[0][0]             
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 768)]        0           tf_distil_bert_model_1[0][0]     
____________________________________________________________________________________________

In [24]:
# fitting model on training data
model.fit([train_input_ids, train_attention_mask], y_train, validation_split=0.2, epochs=10, batch_size=512)

Epoch 1/10
350/350 [==============================] - 118s 337ms/step - accuracy: 0.9359 - loss: 0.1612 - val_accuracy: 0.9369 - val_loss: 0.1420
Epoch 2/10
350/350 [==============================] - 95s 273ms/step - accuracy: 0.9601 - loss: 0.0995 - val_accuracy: 0.9385 - val_loss: 0.1383
Epoch 3/10
350/350 [==============================] - 96s 273ms/step - accuracy: 0.9649 - loss: 0.0872 - val_accuracy: 0.9276 - val_loss: 0.1677
Epoch 4/10
350/350 [==============================] - 96s 273ms/step - accuracy: 0.9691 - loss: 0.0777 - val_accuracy: 0.9372 - val_loss: 0.1444
Epoch 5/10
350/350 [==============================] - 96s 273ms/step - accuracy: 0.9726 - loss: 0.0684 - val_accuracy: 0.9274 - val_loss: 0.1797
Epoch 6/10
350/350 [==============================] - 96s 274ms/step - accuracy: 0.9762 - loss: 0.0591 - val_accuracy: 0.9331 - val_loss: 0.1737
Epoch 7/10
350/350 [==============================] - 96s 273ms/step - accuracy: 0.9804 - loss: 0.0497 - val_accuracy: 0.9348 - v

In [25]:
%%time
val = pd.read_csv('/content/drive/My Drive/Data/Jigsaw Multilingual Toxic Comment Classification/validation.csv')
val_input_ids, val_attention_mask = get_inputs(val['comment_text'], 128)
y_val = val['toxic'].values


CPU times: user 9.31 s, sys: 60.1 ms, total: 9.37 s
Wall time: 9.42 s


In [26]:
model.evaluate([val_input_ids, val_attention_mask], y_val)

250/250 [==============================] - 4s 15ms/step - accuracy: 0.8580 - loss: 0.7542


[0.7541863918304443, 0.8580000400543213]

In [16]:
%%time
test = pd.read_csv('/content/drive/My Drive/Data/Jigsaw Multilingual Toxic Comment Classification/test.csv')
test_input_ids, test_attention_mask = get_inputs(test['content'], 128)


CPU times: user 1min 15s, sys: 563 ms, total: 1min 16s
Wall time: 1min 19s


In [ ]:
test['toxic'] = model.predict([test_input_ids, test_attention_mask], verbose=1)

1995/1995 [==============================] - 34s 17ms/step


In [ ]:
test.to_csv('submission.csv', columns = ['id', 'toxic'], index=False)